# MAP PLOT USING FOLIUM (CHARITY PROJECT)
THIS WAS TO PLOT A GEOGRAPHIC MAP TO SHOW AREAS IN NIGERIA (PORTHARCOURT AND LAGOS) THAT NEEDED OUR VOLUNTEER HOURS, ITEMS NEEDED, ITEMS GIVEN, PROJECTS DONE, PROJECTS ONGOING, ETC.

In [6]:
import json
with open("C:\\Users\\Leo-SJ\\Documents\\MASTERS\\convert (2).json") as f: #loading the converted shapefile in json showing states, their local governments and coordinates
    nigeria_data=json.load(f)
lat=[0]
long=[0]

#Get the average of the longitude and latiutude of each states and their local governments and append to an array "latlong".
latlong=[]

for state in range(len(nigeria_data['features'])):
    total_x=0
    total_y=0
    data=nigeria_data['features'][state]['geometry']['coordinates'][0]
    l=len(data)
    for i in range(l-1):
        total_x+=data[i][0]
        total_y+=data[i][1]
    latlong.append([nigeria_data['features'][state]['properties']['ADM1'],nigeria_data['features'][state]['properties']['ADM2'],total_x/l,total_y/l])
# latlong

In [2]:
nigeria_data['features'][0] #view of nigeria_data structure

{'type': 'Feature',
 'properties': {'ID': 1,
  'LBL': 'NIR-1',
  'FIP': 'NI',
  'MMT_ID': 'NIR',
  'SHORT__FRM': 'Nigeria',
  'LONG_FRM': 'Federal Republic of Nigeria',
  'ADM0': 'Nigeria',
  'ADM1': 'Abia',
  'ADM2': 'Aba',
  'ADM3': '-',
  'ADM4': '-',
  'ADM5': '-',
  'STL-0': 168,
  'STL-1': 1,
  'STL-2': 11,
  'STL-3': '-',
  'STL-4': '-',
  'STL-5': '-'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[7.289999961853027, 5.059999942779541],
    [7.28000020980835, 5.079999923706055],
    [7.28000020980835, 5.090000152587891],
    [7.28000020980835, 5.099999904632568],
    [7.289999961853027, 5.110000133514404],
    [7.300000190734863, 5.110000133514404],
    [7.300000190734863, 5.130000114440918],
    [7.320000171661377, 5.139999866485596],
    [7.329999923706055, 5.150000095367432],
    [7.340000152587891, 5.150000095367432],
    [7.349999904632568, 5.139999866485596],
    [7.360000133514404, 5.139999866485596],
    [7.360000133514404, 5.130000114440918],
    [7.369999885559

In [31]:
#CONVERT ARRAY "latlong" TO A DATAFRAME "df"
import pandas as pd
df=pd.DataFrame(latlong)
df.rename(columns={0:'states',1:'local_govt',2:'long',3:'lat'},inplace=True) #change column names to appropriate ones
df.groupby('states').first().head() #view few data in dataframe

# df.loc[(df['states']=='Rivers') & (df['states'].=='Lagos')]
# df.loc[df['states'].isin(['Rivers','Lagos'])].groupby('states').first()

,local_govt,long,lat
states,,,
Abia,Aba,7.034400,4.894800
Adamawa,Fufore,12.621060,9.116739
Akwa Ibom,Abak,7.590566,4.920000
Anambra,Aguata,6.813333,5.785333
Bauchi,Alkaleri,10.415723,9.858855


In [43]:
#LOADING COMPILED LIST OF SCHOOLS, AREAS, AND ITEMS NEEDED/GIVEN FROM EXCEL TO A DATAFRAME "latlong2" FOR PORTHARCOURT AND LAGOS
latlong2=pd.read_excel('ph_lag.xlsx',na_values=True)
latlong2.groupby('state').first()

,local_govt,lon,lat,TEXTBOOKS,NOTEBOOKS,BAGS,SCHOLARSHIPS,SANDALS,STATIONERY,LOW_VISION_GLASSES,SCHOOL,AMOUNT
state,,,,,,,,,,,,
Lagos,Agege,3.197600,6.386000,TEXTBOOKS,NOTEBOOKS,BAGS,SCHOLARSHIPS,SANDALS,STATIONERY,NaN,NaN,"N50,000"
Rivers,Abua/Odu,6.419636,4.735818,TEXTBOOKS,NOTEBOOKS,BAGS,SCHOLARSHIPS,SANDALS,STATIONERY,"LOWVISIONGLASSES-N300,000","OLANADA SCHOOL 1, OLANADA SCHOOL 2, COMMUNIT...","N200,000"


In [46]:
#SEPARATING AREAS WHERE PROJECTS HAVE BEEN DONE, ONGOING AND NOT STARTED
not_started=[]
started=[]
for lat,lon,state,lg,tb,nb,bg,sch,snd,stat,lvg in zip(latlong2['lat'],latlong2['lon'],latlong2['state'],latlong2['local_govt'],latlong2['TEXTBOOKS'],latlong2['NOTEBOOKS'],latlong2['BAGS'],latlong2['SCHOLARSHIPS'],latlong2['SANDALS'],latlong2['STATIONERY'],latlong2['LOW_VISION_GLASSES']): 
    if type(tb)==float:
        if type(nb)==float:
            if type(bg)==float:
                if type(sch)==float:
                    if type(snd)==float:
                        if type(stat)==float:
                            if type(lvg)==float:
                                not_started.append(lg)
        
not_started
for data in latlong2['local_govt']:
    if data not in not_started:
        started.append(data)
# started=latlong[1]-not_started
started[0:5]

['Agege', 'Epe', 'Eti-Osa', 'Ikorodu', 'Akukutor']

In [45]:
#ADDING A "status" ELEMENT IN THE "nigeria_data" json DATA
for i in range(len(nigeria_data['features'])):
    if nigeria_data['features'][i]['properties']['ADM2'] in not_started:
        nigeria_data['features'][i]['properties']['status']='in_progress'
    elif nigeria_data['features'][i]['properties']['ADM2'] in started:
        nigeria_data['features'][i]['properties']['status']='done'
    else:
        nigeria_data['features'][i]['properties']['status']='not_started'

In [47]:
#USING FOLIUM LIBRARY TO PLOT THE MAP

import folium
from folium import plugins

map=folium.Map(location=[df['lat'].mean(),df['long'].mean()],zoom_start=6.3,tiles='OpenStreetMap') #GIVE COORDINATES OF THE AREA WHICH IS NIGERIA

# items=['TEXTBOOKS','NOTEBOOKS','BAGS','SCHOLARSHIPS','STATIONERY','LVG','SANDAL'] 

m = folium.plugins.MarkerCluster(control=False)
# mcg=folium.FeatureGroup(name='ITEMS GIVEN / NEEDED')
map.add_child(m)

# for i in items:
#     i=folium.plugins.FeatureGroupSubGroup(mcg,str(i))
#     map.add_child(i)

#SPECIFYING DIFFERENT LAYERS IN THE MAP   
TEXTBOOKS=folium.plugins.FeatureGroupSubGroup(m,'TEXTBOOKS')
map.add_child(TEXTBOOKS)

NOTEBOOKS=folium.plugins.FeatureGroupSubGroup(m,'NOTEBOOKS')
map.add_child(NOTEBOOKS)

BAGS=folium.plugins.FeatureGroupSubGroup(m,'BAGS')
map.add_child(BAGS)

SCHOLARSHIPS=folium.plugins.FeatureGroupSubGroup(m,'SCHOLARSHIPS')
map.add_child(SCHOLARSHIPS)

STATIONERY=folium.plugins.FeatureGroupSubGroup(m,'STATIONERY')
map.add_child(STATIONERY)

LVG=folium.plugins.FeatureGroupSubGroup(m,'LOW VISION GLASSES')
map.add_child(LVG)

SANDAL=folium.plugins.FeatureGroupSubGroup(m,'SANDAL')
map.add_child(SANDAL)

#GROUPING THE DIRRERENT ITEMS WITH DIFFERENT COLORS
for lat,lon,state,lg,tb,nb,bg,sch,snd,stat,lvg,amt in zip(latlong2['lat'],latlong2['lon'],latlong2['state'],latlong2['local_govt'],latlong2['TEXTBOOKS'],latlong2['NOTEBOOKS'],latlong2['BAGS'],latlong2['SCHOLARSHIPS'],latlong2['SANDALS'],latlong2['STATIONERY'],latlong2['LOW_VISION_GLASSES'],latlong2['AMOUNT']):
    folium.Circle([lat,lon],radius=40).add_to(map)
    if tb=='TEXTBOOKS':
        folium.Marker(location=[lat,lon],popup=(folium.Popup("@"+lg+" "+str(amt))),icon=folium.Icon(color='Blue',icon_color='Blue')).add_to(TEXTBOOKS)
    if bg=='BAGS':
        folium.Marker(location=[lat,lon],popup=(folium.Popup("@"+lg+" "+str(amt))),icon=folium.Icon(color='Green',icon_color='Green')).add_to(BAGS)
    if nb=='NOTEBOOKS':
        folium.Marker(location=[lat,lon],popup=(folium.Popup("@"+lg+" "+str(amt))),icon=folium.Icon(color='Black',icon_color='Black')).add_to(NOTEBOOKS)
    if stat=='STATIONERY':
        folium.Marker(location=[lat,lon],popup=(folium.Popup("@"+lg+" "+str(amt))),icon=folium.Icon(color='Orange',icon_color='Orange')).add_to(STATIONERY)
    if sch=='SCHOLARSHIPS':
        folium.Marker(location=[lat,lon],popup=(folium.Popup("@"+lg+" "+str(amt))),icon=folium.Icon(color='Purple',icon_color='Purple')).add_to(SCHOLARSHIPS)
    if str(lvg).find('LOWVISIONGLASSES')==0:
        folium.Marker(location=[lat,lon],popup=(folium.Popup("@"+lg+" "+lvg)),icon=folium.Icon(color='Yellow',icon_color='Yellow')).add_to(LVG)
    if lvg=='SANDAL':
        folium.Marker(location=[lat,lon],popup=(folium.Popup("@"+lg+" "+str(amt))),icon=folium.Icon(color='White',icon_color='White')).add_to(SANDAL)
#     fg.add_child(folium.Marker(location=[lat,lon],popup=(folium.Popup(lg)),icon=folium.Icon(color=color(state),icon_color='red')))


# style="position: fixed;
# bottom: 50px;
# left: 50px;
# width: 150px;
# height: 280px;
# border: 2px solid grey;
# z-index: 9999;

#ADDING A LEGEND TO THE MAP IN HTML TAGS
lgnd_html='''
<div style="position:relative;
width:50%
border: 5px solid grey;
">
<div 
style="position: absolute;
bottom: 50px;
left: 50px;
width: 100%;
height: 100%;
border: 5px solid grey;
z-index: 9999;"> 

&nbsp; <strong style="text-color:white;">ITEMS LEGEND</strong> <br> <br><br> &nbsp;

<strong>TEXTBOOKS</strong> &nbsp;
<i class = "fa-rotate-0 glyphicon glyphicon-info-sign fa-1x"
style="color:blue"></i>
<br>&nbsp;

<strong>BAGS</strong> &nbsp; 
<i class = "fa-rotate-0 glyphicon glyphicon-info-sign"
style="color:green"></i>
<br>&nbsp;

<strong>NOTEBOOKS</strong> &nbsp;
<i class = "fa-rotate-0 glyphicon glyphicon-info-sign"
style="color:black"></i>
<br>&nbsp;

<strong>STATIONERY</strong> &nbsp; 
<i class = "fa-rotate-0 glyphicon glyphicon-info-sign"
style="color:orange"></i>
<br>&nbsp;

<strong>SCHOLARSHIPS</strong> &nbsp;
<i class = "fa-rotate-0 glyphicon glyphicon-info-sign"
style="color:purple"></i>
<br>&nbsp; 

<strong>LOW VISION GLASSES</strong> &nbsp; 
<i class = "fa-rotate-0 glyphicon glyphicon-info-sign"
style="color:yellow"></i>
<br>&nbsp;

<strong>SANDALS</strong> &nbsp; 
<i class = "fa-rotate-0 glyphicon glyphicon-info-sign"
style="color:white"></i>
</div></div>'''

#ADD HTML LEGEND TO MAP LAYER
map.get_root().html.add_child(folium.Element(lgnd_html))

# fg2=folium.FeatureGroup(name="Bags")

# <div class="awesome-marker-icon-Green awesome-marker leaflet-zoom-animated leaflet-interactive" tabindex="0" style="margin-left: -17px; margin-top: -42px; width: 35px; height: 45px; opacity: 1; transform: translate3d(598px, 640px, 0px); z-index: 1000640;"><i style="color: Green" class="fa-rotate-0 glyphicon glyphicon-info-sign  "></i></div>
# <i class="fa fa-map-marker fa-2x" style="color:green"></i>
# map.add_child(fg)
# map.add_child(fg2)


#GROUPING COLORS FOR THE DIFFERENT AREAS SHOWING DONE, ONGOING AND NOT STARTED PROJECTS
map.add_child(folium.GeoJson(data=nigeria_data,
                             name="DONE",overlay=False,
                             style_function=lambda x:{'fillColor':
                                 'green' if x['properties']['status']=='done' else
                                 'white'                   
                             }).add_to(map))
map.add_child(folium.GeoJson(data=nigeria_data,
                             name="ON GOING",overlay=False,
                             style_function=lambda x:{'fillColor':
                                 'yellow' if x['properties']['status']=='in_progress' else
                                'white'                    
                             }))
map.add_child(folium.GeoJson(data=nigeria_data,
                             name="NOT STARTED",overlay=False,
                             style_function=lambda x:{'fillColor':
                                 'red' if x['properties']['status']=='not_started' else
                                  'white'
                             }))
folium.LayerControl(collapsed=False).add_to(map)
# map.add_child(folium.LayerControl(collapsed=False))
map.save(outfile='rcm.html')   #SAVE MAP IN HTML EXTENSION

# DATA VALIDATION AT PETRODATA (BACKEND DATA UPLOAD WRANGLING)
DURING MY NATIONAL YOUTH SERVICE AT PETRODATA MANAGEMENT LIMITED SERVICES THERE WAS A PROJECT TO UPGRADE THEIR DATA MANAGEMENT SOLUTION AND CLEAN UP THE COMPANY'S BACK-END DATABASE FOR THEIR OIL AND GAS CLIENTS. I HELPED IN CHANGING THE DATA FORMATS ESPECIALLY THE DATE FORMATS AND TO ALSO COMPARE WITH SOME OF THE PHYSICAL STORAGE MEDIA. THE TOTAL DATA WORKED WITH CONTAINED OVER A HUNDRED THOUSAND RECORDS.

In [1]:
#LOADING PYTHON LIBRARIES
import os
import pandas as pd
import pprint
import csv
import collections
import numpy as np
os.getcwd()

'C:\\Users\\Leo-SJ\\Desktop\\DATA VALIDATION\\CODE'

In [2]:
#SETTING CLIENT'S UNIQUE ID
filename='AD01' 
filename_concat=filename+'__'
ext='.xlsx'

# READING FILES

In [7]:
#LOADING DATA

os.chdir('C:\\Users\\Leo-SJ\\Desktop\\DATA VALIDATION')
# # os.chdir('E:\DATA VALIDATION')
path='C:\\Users\\Leo-SJ\\Desktop\\DATA VALIDATION\\'
# os.chdir('C:\\Users\\oluwaseun.johnson\\Desktop\\DATA VALIDATION')
# path='C:\\Users\\oluwaseun.johnson\\Desktop\\DATA VALIDATION\\'
# path_source='Z:\\Mr Biola\\CLIENT Merge data\\'
filename_source_original='Copy of CLIENT DATA LISTING.xlsx'
filename_source_uploaded='uploadedexport.xlsx'
filename_source_extra='Copy of CLIENT MERGING.xlsx'

# data_source_original=pd.read_excel(path_source+filename_source_original)
# data_source_uploaded=pd.read_excel(path_source+filename_source_uploaded)
# data_source_extra=pd.read_excel(path_source+filename_source_extra)

data_original=pd.read_excel(filename_source_original,keep_default_na=False,dtype=str)

data_uploaded=pd.read_excel(filename_source_uploaded,keep_default_na=False)

data_extra=pd.read_excel(filename_source_extra,keep_default_na=False)

In [5]:
data_original.shape  #DIMENSION OFTHE LOADED DATA

(136348, 29)

In [9]:
data_original.columns   #THE FIELD NAMES IN THE DATA

Index(['S/N', 'ClientCode', 'BatchNo', 'MediaNo', 'ProspectName',
       'ConcessionNo', 'DataType', 'LineNumSeq', 'StorageMedium', 'FormatType',
       'DataLength', 'SampleRate', 'ShotPointRange', 'FieldRange',
       'CrossLineRange', 'InLineRange', 'AcqMode', 'AcqContractor', 'AcqDate',
       'ProContractor', 'ProDate', 'DateReceived', 'TapeLocation', 'EntryBy',
       'EntryDate', 'TapeAvailability', 'RetrievalDates', 'ReturnDates',
       'Remarks'],
      dtype='object')

In [10]:
data_original.groupby('ClientCode').first() #FEW DATA GROUPED BY THE "ClientCode"

,S/N,BatchNo,MediaNo,ProspectName,ConcessionNo,DataType,LineNumSeq,StorageMedium,FormatType,DataLength,...,ProContractor,ProDate,DateReceived,TapeLocation,EntryBy,EntryDate,TapeAvailability,RetrievalDates,ReturnDates,Remarks
ClientCode,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,...,,,,,,,,,,
AD01,1,1,F45008,OWERRI,OPL 118,FIELD,SWATH 1,3480,N/A,N/A,...,PULSONIC,N/A,N/A,A-8-A-169,tony,2001-02-20 00:00:00,AVAILABLE,,,N/A
AE47,1,1,DX0043,ODEAMA,OML 29,DEMUX,N/A,3592,SEGY,N/A,...,N/A,N/A,2015-11-11 00:00:00,D-11-G-090,Adams,2015-11-11 00:00:00,AVAILABLE,,,"INPUT TAPES: I06315, I06316, I06317, I06318, I..."
AM03,1,1,1,N/A,OML 112/117,FIELD,156,3490E,SEGD,4Sec,...,N/A,N/A,2008-02-15 00:00:00,E-11-M-105,IFEANYI,2008-02-21 00:00:00,AVAILABLE,,,
CV06,1,CV06/01,39135,OPL247,OPL247,FIELD,N06A1952J4S117,3590,SEGD,N/A,...,N/A,N/A,N/A,A --6 --P --137,akinlawon,2007-01-24 00:00:00,AVAILABLE,,,FILES:1421-1562
EF14,14812,EF14/1,R60663,GTB - OWOWO,OPL 223,FINAL KPSTM ANGLE STACKS,N/A,3592,SEGY,N/A,...,IDSL/WESTERNGECO NIGERIA,2013-04-01 00:00:00,2014-02-07 00:00:00,G-14-A-9,Adams,2014-02-28 00:00:00,AVAILABLE,,,
ER44,1,1,DX0253,HUGHES,OML 18,CFIELD,N/A,3592,SEGD,N/A,...,PMSL,2014-12-01 00:00:00,2015-08-06 00:00:00,D-11-E-41,CALEB,2015-08-19 00:00:00,AVAILABLE,,,"TAPE_ ID :I05953 - 54, 59 - 60"
ES13,1,ES13/1,A019734,NIGERIA BLOCK 209,OPL 209,FIELD,2051 - 2032,3480,N/A,N/A,...,N/A,N/A,2003-12-04 00:00:00,B-11-C-1,tony,2003-12-04 00:00:00,AVAILABLE,,,NIGERIA BLOCK 209
FH15,88654,1,D04453,3D-91-OGINI,OML 26,CFIELD,NA,3590E,SEGD,N/A,...,N/A,N/A,2012-11-29 00:00:00,F-17-A-32,FALAYI,2017-08-04 00:00:00,AVAILABLE,,,


In [11]:
#FILTER DATA BY EACH CLIENT CODES

original_extract=data_original.loc[data_original['ClientCode']==filename,'MediaNo'].values
upload_extract=data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo'].values
extra_extract=data_extra.loc[data_extra['ClientCode']==filename,'MediaNo'].values
# len(original_extract)
# data_original.head()
# data_uploaded.loc[data_uploaded['ClientCode']==filename,'ProDate']
# data_original.loc[(data_original['ClientCode']==filename) & (data_original['MediaNo']) & (data_original['StorageMedium'])]['StorageMedium']

In [14]:
original_extract

array(['F45008', 'F45004', 'F45005', ..., 'TR-10266-TR-00504',
       'F9497A001A1-F9497A004A', 'R53252-R53254'], dtype=object)

# WORKING ON THE DATE FORMATS

In [15]:
#LOADING EACH DATA BY THEIR CLIENT CODES AND SETTING THE DATA TYPE OF ALL THE DATE FIELDS TO STRINGS

os.chdir('C:\\Users\\Leo-SJ\\Desktop\\DATA VALIDATION\\CORRECTED')
# # os.chdir('E:\DATA VALIDATION')
path='C:\\Users\\Leo-SJ\\Desktop\\DATA VALIDATION\\CORRECTED\\'

# os.chdir('C:\\Users\\oluwaseun.johnson\\Desktop\\DATA VALIDATION\\CORRECTED')
# path='C:\\Users\\oluwaseun.johnson\\Desktop\\DATA VALIDATION\\CORRECTED\\'

filename='AD01' 
filename_concat=filename+'__'
ext='.xlsx'

# data=pd.read_excel(filename+ext,keep_default_na=False,parse_dates=True,index_col=18)
data=pd.read_excel(filename+ext,keep_default_na=False,dtype={'AcqDate':'str','ProDate':'str','DateReceived':'str','EntryDate':'str','RetrievalDates':'str','ReturnDates':'str'}) #change data type oF column acqdate to string
# data=pd.read_excel(filename+ext,keep_default_na=False)

In [19]:
#PROPER DATE FORMATS SHOULD BE IN THE FORM "YYYY-MM-DD 00:00:00" BUT SOME WERE IN THE FORMAT "20100111" AND "41148"
data.groupby('AcqDate').first()

,S/N,ClientCode,BatchNo,MediaNo,ProspectName,ConcessionNo,DataType,LineNumSeq,StorageMedium,FormatType,...,ProContractor,ProDate,DateReceived,TapeLocation,EntryBy,EntryDate,TapeAvailability,RetrievalDates,ReturnDates,Remarks
AcqDate,,,,,,,,,,,,,,,,,,,,,
14/10/2002,2333,AD01,13,TR-9250B-123448,LANKAHUASA SOUTH,OPL 225,FIELD,LS3D1635P043,3590,SEGD,...,N/A,N/A,41148,F-16-H-27,ABIOLA,41158,AVAILABLE,,,NFH/AUX/SEIS:36/0/1120
20/11/2002,2445,AD01,13,TR-9250B-123701,LANKAHUASA SOUTH,OPL 225,FIELD,LS3D1475P106,3590,SEGD,...,N/A,N/A,41148,F-16-J-25,ABIOLA,41159,AVAILABLE,,,NFH/AUX/SEIS:36/0/1120
2000-03-28 00:00:00,512,AD01,2,10,OPL98,OPL 98,2D SEISMIC DATA,N/A,CD,N/A,...,N/A,N/A,N/A,H-4-4-4,tony,2005-06-03 00:00:00,AVAILABLE,,,STORED IN BULK STORAGE AS 222/02/CDs
2000-03-29 00:00:00,510,AD01,2,8,OPL98,OPL 98,2D SEISMIC DATA,N/A,CD,N/A,...,N/A,N/A,N/A,H-4-4-4,tony,2005-06-03 00:00:00,AVAILABLE,,,STORED IN BULK STORAGE AS 222/02/CDs
2000-03-30 00:00:00,511,AD01,2,9,OPL98,OPL 98,2D SEISMIC DATA,N/A,CD,N/A,...,N/A,N/A,N/A,H-4-4-4,tony,2005-06-03 00:00:00,AVAILABLE,,,STORED IN BULK STORAGE AS 222/02/CDs
2000-04-05 00:00:00,520,AD01,2,2,OPL98/90/225,OPL 98,VSP DATA,N/A,CD,N/A,...,N/A,N/A,N/A,H-4-4-4,tony,2005-06-03 00:00:00,AVAILABLE,,,THESE ARE IN BULK STORAGE AS 222/02/CDs
2000-04-11 00:00:00,503,AD01,2,3,OPL90/225,OPL 90,2D SEISMIC DATA,N/A,CD,N/A,...,N/A,N/A,N/A,H-4-4-4,tony,2005-06-03 00:00:00,AVAILABLE,,,THESE ARE IN BULK STORAGE AS 222/02/CDs
2000-04-12 00:00:00,507,AD01,2,0,OPL90/225,OPL 90,CD DATABASE,N/A,CD,N/A,...,N/A,N/A,N/A,H-4-4-4,tony,2005-06-03 00:00:00,AVAILABLE,,,THESE ARE IN BULK STORAGE AS 222/02/CDs
2000-04-17 00:00:00,508,AD01,2,18,OPL90/98/118/225,OPL 98,CHECK SHOT DATA,N/A,CD,N/A,...,N/A,N/A,N/A,H-4-4-4,tony,2005-06-03 00:00:00,AVAILABLE,,,STORED IN BULK STORAGE AS 222/02/CDs


In [22]:
#IF DATE FORMAT LIKE "41148" LOOP THROUGH AND CHANGE ALL DATE FORMAT TO A TIME STAMP IN FORMAT "YYYY-MM-DD 00:00:00"
import datetime as DT
count=0
for item in data['AcqDate'].values:
    if len(str(data['AcqDate'].iloc[count]))==5:
        data['AcqDate'][count]=pd.to_datetime(DT.date(1900,1,1) + DT.timedelta(days=int(data['AcqDate'].iloc[count])-2))
        count=count+1
    else:
        count=count+1
# date=data['AcqDate']


C:\Users\Leo-SJ\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
#IF DATE FORMAT LIKE "20100111" LOOP THROUGH AND CHANGE ALL DATE FORMAT TO A TIME STAMP IN FORMAT "YYYY-MM-DD 00:00:00"
count=0
for item in data['AcqDate'].values:    
    if len(str(data['AcqDate'].iloc[count]))==8:
        data['AcqDate'][count]=pd.to_datetime(data['AcqDate'].iloc[count])
        count=count+1
    else:
        count=count+1

C:\Users\Leo-SJ\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [25]:
#CHANGE ALL DATE FORMATS TO TIME STAMP "YYYY-MM-DD 00:00:00"
count=0
# for count in range(0,len(data['AcqDate'].values)):
for item in data['AcqDate'].values:
#     if type(item)==str:
#         count=count+1
    data['AcqDate'][count]=pd.to_datetime(data['AcqDate'].iloc[count],errors='coerce')
    count=count+1
#     else:
#         count=count+1

C:\Users\Leo-SJ\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [26]:
#SAVE DATAFRAME INTO ANOTHER EXCEL FILE
pd.DataFrame(data).to_excel('C:\\Users\\Leo-SJ\\Desktop\\DATA VALIDATION\\CORRECTED\\DATE_.xlsx',header=True,index=False) #changes array to dataframe 1st then to excel

'C:\\Users\\Leo-SJ\\Desktop\\DATA VALIDATION\\CORRECTED'

# OTHER PLAY AROUND WITH THE DATA

In [179]:
data['EntryDate'].iloc[0]

'2003-12-04 00:00:00'

In [19]:
data['AcqDate'][0]

'39385'

In [25]:
data['AcqDate'][0]=pd.to_datetime(DT.date(1900,1,1) + DT.timedelta(days=int(data['AcqDate'].iloc[0])-2))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [85]:
count = 0
for items in data['AcqDate'].values:   
    if len(str(items))==5:      
        print(items,count)
        count=count+1
    else:
        count=count+1

In [209]:
count=0
for item in data['RetrievalDates'].values:
    
#     if len(str(item)) <5:
#         item = 0
#         print(item)
#     print(item)
#     if type(data['AcqDate'].iloc[count])==int:
    if len(str(data['RetrievalDates'].iloc[count]))==21:
        data['RetrievalDates'][count]=pd.to_datetime(' '.join(data['RetrievalDates'].iloc[count].split(';',1)[1].split()))
        count=count+1
    else:
        count=count+1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


IndexError: single positional indexer is out-of-bounds

In [16]:
for item in data.groupby('AcqDate')['S/N'].nunique().values:
    print(item)

1
1
1
5
2
1
8
1
1
3
122
8
4
10
5
11
1
2
6
6
11
4
6
3
14
4
7
9
3
3
1
10
1
8
3
1
6
2
20
3
12
5
8
5
4
7
4
9
3
10
1
5
3
7
2
7
5
5
4
4
6
10
5
7
5
1
4
4
9
7
9
5
8
3
2
8
4
1
2
11
9
22
18
9
5
11
14
19
4
27
93
1
1
2
3
3
1
2
5
9
11
11
9
5
9
6
10
6
4
5
10
5
1
8
11
7
9
4
2
18
285
1
123
1
8
10
11
8
8
8
9
10
8
9
4
71
4
2
2
12
8
7
6
7
6
8
7
5
17
8
9
6
7
8
7
9
8
10
6
9
9
6
7
5
7
9
6
5
6
7
9
5
7
4
9
23
12
14
15
12
14
11
9
12
10
12
13
11
10
12
14
13
14
22
1
27
13
1
1
1
3
8
4
7
4
12
19
18
36
27
13
10
18
20
25
7
2
2
3
5
5
5
1
2
9
2
8
4
1
8
1
1
1
2
2
1
1
3
1
3
1
3
2
1
1
2
2
2
4
2
3
6
2
2
1
2
4
1
4
5
1
1
1
1
1
3
3
1
3
2
3
4
4
6
5
6
3
7
4
7
3
1
93
130


In [145]:
data['ProDate']='NULL'

In [ ]:
# AcqDate':'str','ProDate':'str','DateReceived':'str','EntryDate':'str','RetrievalDates':'str','ReturnDates':'str'

In [121]:
from dateutil import parser
# data.AcqDate.apply(lambda x: parser.parse(x))
# date=pd.to_datetime(data.AcqDate,format='%Y-%j')  #'coerce','raise','ignore'
# date=date.astype(str)
date=pd.to_datetime(data['DateReceived'],errors='coerce')
# type(data.AcqDate.astype(str))
type(date)

pandas.core.series.Series

FINAL GUY!!

In [59]:
for item in s:
    if "DATE" in item:
        print(item)

In [83]:
count=19968
for item in data['TapeLocation'].values:    
#     if len(str(data['Remarks'].iloc[count]))>255:
#     if "DREATED" in str(data['Remarks'].iloc[count]):
    if '-- -' in str(data['TapeLocation'].iloc[count]):
#         data['Remarks'][count]=pd.to_datetime(data['Remarks'].iloc[count])
        print(data['TapeLocation'].iloc[count],count)
#         data['Remarks'][count]=' '.join(data['Remarks'].iloc[count].split('DATE',1)[0].split())
        count=count+1
    else:
        count=count+1

 -- -- -- 19968
 -- -- -- 19970
 -- -- -- 19971
 -- -- -- 19973
 -- -- -- 19974
 -- -- -- 19976
 -- -- -- 19978
 -- -- -- 19979
 -- -- -- 19981
 -- -- -- 19982
 -- -- -- 19983
 -- -- -- 19984
 -- -- -- 19986
 -- -- -- 19995
 -- -- -- 19997
 -- -- -- 19999
 -- -- -- 20001
 -- -- -- 20003
 -- -- -- 20004
 -- -- -- 20006
 -- -- -- 20008
 -- -- -- 20010
 -- -- -- 20011
 -- -- -- 20013
 -- -- -- 20014
 -- -- -- 20016
 -- -- -- 20018
 -- -- -- 20020
 -- -- -- 20022
 -- -- -- 20023
 -- -- -- 20025
 -- -- -- 20027
 -- -- -- 20029
 -- -- -- 20031
 -- -- -- 20032
 -- -- -- 20034
 -- -- -- 20036
 -- -- -- 20038
 -- -- -- 20040
 -- -- -- 20042
 -- -- -- 20044
 -- -- -- 20046
 -- -- -- 20048
 -- -- -- 20050
 -- -- -- 20051
 -- -- -- 20053
 -- -- -- 20055
 -- -- -- 20056
 -- -- -- 20058
 -- -- -- 20060
 -- -- -- 20062
 -- -- -- 20064
 -- -- -- 20066
 -- -- -- 20068
 -- -- -- 20070
 -- -- -- 20072
 -- -- -- 20073
 -- -- -- 20074
 -- -- -- 20075
 -- -- -- 20076
 -- -- -- 20077
 -- -- -- 20079
 -- -- -

In [94]:

# data.filter(like='Date') #filter only columns that have the value Date in them
# data.filter(like='Date').columns.values 
# data.filter(like='Date').apply(str)
# data.columns.get_loc('AcqDate') #get index number of column AcqDate
# data.TapeAvailability
# data.columns.values
data[data['MediaNo']=='TR-10006'][['EntryBy','TapeLocation']]
# data2=data[data['TapeAvailability']!='NOT AVAILABLE'][['AcqDate','S/N']]
# data2

,EntryBy,TapeLocation
773,akin,A-10-M-156


In [20]:
#

s=[]
sj=[]
for item in data['AcqDate'].values: #appends all the  data type in an array
    print(item,type(item))
#     if len(str(item))>255:
    s.append((item,type(item),len(str(item))))
for item in data['AcqDate'].values: #appends all the length of the differnt data type in another array
    print(item,type(item))
    sj.append(len(str(item))) 
    
# for item, count in collections.Counter(s).items():  #returns the datatypes and the count
#     print(item,count) 
# for item, count in collections.Counter(sj).items(): #returns the count of the lengths
#     print(item)

# for item in data['AcqDate'].values:  #checks the length and changes the data type
#     if type(item) == int:
#         if len(str(item))>5:
#             item=str(item)
# s
# print(filter(lambda x: type(x)==str,data['AcqDate'].values))
s

2000-07-02 00:00:00 <class 'str'>
2000-06-30 00:00:00 <class 'str'>
2000-07-02 00:00:00 <class 'str'>
2000-07-02 00:00:00 <class 'str'>
2000-07-02 00:00:00 <class 'str'>
2000-07-02 00:00:00 <class 'str'>
2000-06-30 00:00:00 <class 'str'>
2000-07-02 00:00:00 <class 'str'>
2000-07-02 00:00:00 <class 'str'>
2000-07-03 00:00:00 <class 'str'>
2000-07-03 00:00:00 <class 'str'>
2000-07-03 00:00:00 <class 'str'>
2000-07-04 00:00:00 <class 'str'>
2000-07-04 00:00:00 <class 'str'>
2000-07-03 00:00:00 <class 'str'>
2000-06-30 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:00:00 <class 'str'>
2000-07-01 00:

2002-01-20 00:00:00 <class 'str'>
2002-01-20 00:00:00 <class 'str'>
2002-01-20 00:00:00 <class 'str'>
2002-01-20 00:00:00 <class 'str'>
2002-01-21 00:00:00 <class 'str'>
2002-01-21 00:00:00 <class 'str'>
2002-01-21 00:00:00 <class 'str'>
2002-01-21 00:00:00 <class 'str'>
2002-01-21 00:00:00 <class 'str'>
2002-01-21 00:00:00 <class 'str'>
2002-01-21 00:00:00 <class 'str'>
2002-01-21 00:00:00 <class 'str'>
2002-01-21 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:00:00 <class 'str'>
2002-01-22 00:

2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:

20090123 <class 'str'>
20090123 <class 'str'>
20090123 <class 'str'>
20090123 <class 'str'>
20090124 <class 'str'>
20090124 <class 'str'>
20090124 <class 'str'>
20090124 <class 'str'>
20090124 <class 'str'>
20090125 <class 'str'>
20090125 <class 'str'>
20090125 <class 'str'>
20090125 <class 'str'>
20090126 <class 'str'>
20090126 <class 'str'>
20090125 <class 'str'>
20081127 <class 'str'>
20081126 <class 'str'>
20081126 <class 'str'>
20081126 <class 'str'>
20081126 <class 'str'>
20081127 <class 'str'>
20081127 <class 'str'>
20081128 <class 'str'>
20081128 <class 'str'>
20081129 <class 'str'>
20081129 <class 'str'>
20081130 <class 'str'>
20081130 <class 'str'>
20081130 <class 'str'>
20081201 <class 'str'>
20081202 <class 'str'>
20081202 <class 'str'>
20081202 <class 'str'>
20081203 <class 'str'>
20081203 <class 'str'>
20081203 <class 'str'>
20081204 <class 'str'>
20081205 <class 'str'>
20081205 <class 'str'>
20081205 <class 'str'>
20081205 <class 'str'>
20081205 <class 'str'>
20081209 <c

20060108 <class 'str'>
20060108 <class 'str'>
20060108 <class 'str'>
20060108 <class 'str'>
20060108 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060111 <class 'str'>
20060111 <class 'str'>
20060111 <class 'str'>
20060111 <class 'str'>
20101107 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <c

...

2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-02-28 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:00:00 <class 'str'>
2006-09-01 00:

20090123 <class 'str'>
20090124 <class 'str'>
20090124 <class 'str'>
20090124 <class 'str'>
20090124 <class 'str'>
20090124 <class 'str'>
20090125 <class 'str'>
20090125 <class 'str'>
20090125 <class 'str'>
20090125 <class 'str'>
20090126 <class 'str'>
20090126 <class 'str'>
20090125 <class 'str'>
20081127 <class 'str'>
20081126 <class 'str'>
20081126 <class 'str'>
20081126 <class 'str'>
20081126 <class 'str'>
20081127 <class 'str'>
20081127 <class 'str'>
20081128 <class 'str'>
20081128 <class 'str'>
20081129 <class 'str'>
20081129 <class 'str'>
20081130 <class 'str'>
20081130 <class 'str'>
20081130 <class 'str'>
20081201 <class 'str'>
20081202 <class 'str'>
20081202 <class 'str'>
20081202 <class 'str'>
20081203 <class 'str'>
20081203 <class 'str'>
20081203 <class 'str'>
20081204 <class 'str'>
20081205 <class 'str'>
20081205 <class 'str'>
20081205 <class 'str'>
20081205 <class 'str'>
20081205 <class 'str'>
20081209 <class 'str'>
20081209 <class 'str'>
20081210 <class 'str'>
20081210 <c

20060108 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060109 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060110 <class 'str'>
20060111 <class 'str'>
20060111 <class 'str'>
20060111 <class 'str'>
20060111 <class 'str'>
20101107 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <class 'str'>
20101120 <c

N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class 'str'>
N/A <class

[('2000-07-02 00:00:00', str, 19),
 ('2000-06-30 00:00:00', str, 19),
 ('2000-07-02 00:00:00', str, 19),
 ('2000-07-02 00:00:00', str, 19),
 ('2000-07-02 00:00:00', str, 19),
 ('2000-07-02 00:00:00', str, 19),
 ('2000-06-30 00:00:00', str, 19),
 ('2000-07-02 00:00:00', str, 19),
 ('2000-07-02 00:00:00', str, 19),
 ('2000-07-03 00:00:00', str, 19),
 ('2000-07-03 00:00:00', str, 19),
 ('2000-07-03 00:00:00', str, 19),
 ('2000-07-04 00:00:00', str, 19),
 ('2000-07-04 00:00:00', str, 19),
 ('2000-07-03 00:00:00', str, 19),
 ('2000-06-30 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:00', str, 19),
 ('2000-07-01 00:00:

In [89]:
data['AcqDate'] = data['AcqDate'].map(lambda x: x.strip())

In [95]:
# pd.unique(data['StorageMedium'])
pd.options.display.max_rows = None
pd.options.display.max_columns = None
data2.groupby('AcqDate')['S/N'].nunique()
# data.groupby('AcqDate')['S/N'].nunique()


AcqDate
14/10/2002               1
20/11/2002               1
2000-03-28 00:00:00      1
2000-03-29 00:00:00      5
2000-03-30 00:00:00      2
2000-04-05 00:00:00      1
2000-04-11 00:00:00      8
2000-04-12 00:00:00      1
2000-04-17 00:00:00      1
2000-06-30 00:00:00      3
2000-07-01 00:00:00    122
2000-07-02 00:00:00      8
2000-07-03 00:00:00      4
2000-07-04 00:00:00     10
2000-07-05 00:00:00      5
2000-07-06 00:00:00     11
2000-07-07 00:00:00      1
2000-07-10 00:00:00      2
2000-07-11 00:00:00      6
2000-07-12 00:00:00      6
2000-07-13 00:00:00     11
2000-07-14 00:00:00      4
2000-07-15 00:00:00      6
2000-07-16 00:00:00      3
2000-07-17 00:00:00     14
2000-07-18 00:00:00      4
2000-07-25 00:00:00      7
2000-07-26 00:00:00      9
2000-07-27 00:00:00      3
2000-08-01 00:00:00      3
2000-08-03 00:00:00      1
2000-08-04 00:00:00     10
2000-08-05 00:00:00      1
2000-08-06 00:00:00      8
2000-08-07 00:00:00      3
2000-08-08 00:00:00      1
2000-08-09 00:00:00 

In [89]:
locate=[]
count=19968
for item in data['TapeLocation'].values:
#     print(data['TapeLocation'].iloc[count])
    locate.append(data['TapeLocation'].iloc[count])
    count=count+1

IndexError: single positional indexer is out-of-bounds

In [92]:
data[(data['TapeLocation'].isin(locate))]

,S/N,ClientCode,BatchNo,MediaNo,ProspectName,ConcessionNo,DataType,LineNumSeq,StorageMedium,FormatType,...,ProContractor,ProDate,DateReceived,TapeLocation,EntryBy,EntryDate,TapeAvailability,RetrievalDates,ReturnDates,Remarks
3828,3829,CV06,CV06/13,20729003,OLOMU,OML95,Field,,8MM,,...,,,,B --1 --A --29,Olomu 1,,NOT AVAILABLE,2005-07-06 00:00:00,,Olomu 1
3831,3832,CV06,CV06/13,20729006,OLOMU,OML95,Field,,8MM,,...,,,,B --1 --A --42,Olomu 1,,NOT AVAILABLE,2005-07-06 00:00:00,,Olomu 1
3833,3834,CV06,CV06/13,20735001,OLOMU,OML95,Field,,8MM,,...,,,,B --1 --A --41,Olomu 1,,NOT AVAILABLE,,,Olomu 1
3837,3838,CV06,CV06/13,933,OLOMU-1,OML95,P190 NAVIGATION,95TD-240PI,8MM,,...,PULSONIC NIGERIA LTD.,,,B --12 --E --70,,,NOT AVAILABLE,2005-07-06 00:00:00,,
4432,4433,CV06,CV06/15,PMSL-023,OLOMU,OML95,Migration,N/A,8MM,SEG-Y,...,WESTERN GEOPHYSICAL,,,B --1 --A --30,Migrated stacks; 20 files each seperated by si...,,AVAILABLE,,,Migrated stacks; 20 files each seperated by si...
18379,18380,CV06,CV06/51,DL9947,OPL215,OPL215,Raw Full Stack Post PSTM,N/A,DLT,SEGY,...,CGG,,2007-12-29 00:00:00,N/A --N/A --N/A --N/A,Rec.Length: 8.5s; No Storage Slot before Retr...,,NOT AVAILABLE,2008-04-14 00:00:00,,Rec.Length: 8.5s; No Storage Slot before Retr...
18380,18381,CV06,CV06/51,DL9948,OPL215,OPL215,Raw Full Stack Post PSTM,N/A,DLT,SEGY,...,CGG,,2007-12-29 00:00:00,N/A --N/A --N/A --N/A,Rec.Length: 8.5s; No Storage Slot before Retr...,,NOT AVAILABLE,2008-04-14 00:00:00,,Rec.Length: 8.5s; No Storage Slot before Retr...
18381,18382,CV06,CV06/51,DL9949,OPL215,OPL215,Raw Full Stack Post PSTM,N/A,DLT,SEGY,...,CGG,,2007-12-29 00:00:00,N/A --N/A --N/A --N/A,Rec.Length: 8.5s; No Storage Slot before Retr...,,NOT AVAILABLE,2008-04-14 00:00:00,,Rec.Length: 8.5s; No Storage Slot before Retr...
18382,18383,CV06,CV06/51,DL9950,OPL215,OPL215,Raw Full Stack Post PSTM,N/A,DLT,SEGY,...,CGG,,2007-12-29 00:00:00,N/A --N/A --N/A --N/A,Rec.Length: 8.5s; No Storage Slot before Retr...,,NOT AVAILABLE,2008-04-14 00:00:00,,Rec.Length: 8.5s; No Storage Slot before Retr...
18383,18384,CV06,CV06/51,DL9951,OPL215,OPL215,Raw Full Stack Post PSTM,N/A,DLT,SEGY,...,CGG,,2007-12-29 00:00:00,N/A --N/A --N/A --N/A,Rec.Length: 8.5s; No Storage Slot before Retr...,,NOT AVAILABLE,2008-04-14 00:00:00,,Rec.Length: 8.5s; No Storage Slot before Retr...


In [223]:
var='             '
' '.join(var.split())

''

In [73]:
# data['Remarks'].iloc[22474]
' '.join(data['Remarks'].iloc[22474].split('DATE',1)[0].split())

'1 INPUT MEDIA, TAPE ID: NL4056; BYTE 181: XCORD CELL CENTER, BYTE 185: XCORD CELL CENTER, BYTE 189: 3DT SEC ORD CELCTR, BYTE 193: 3DT PRIM ORD CELCTR;'

In [90]:
count = 0
for items in data['ReturnDates'].values:   
    if '' in item:      
        print(items,count)
        count=count+1
    else:
        count=count+1

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31
 32
 33
 34
 35
 36
 37
 38
 39
 40
 41
 42
 43
 44
 45
 46
 47
 48
 49
 50
 51
 52
 53
 54
 55
 56
 57
 58
 59
 60
 61
 62
 63
 64
 65
 66
 67
 68
 69
 70
 71
 72
 73
 74
 75
 76
 77
 78
 79
 80
 81
 82
 83
 84
 85
 86
 87
 88
 89
 90
 91
 92
 93
 94
 95
 96
 97
 98
 99
 100
 101
 102
 103
 104
 105
 106
 107
 108
 109
 110
 111
 112
 113
 114
 115
 116
 117
 118
 119
 120
 121
 122
 123
 124
 125
 126
 127
 128
 129
 130
 131
 132
 133
 134
 135
 136
 137
 138
 139
 140
 141
 142
 143
 144
 145
 146
 147
 148
 149
 150
 151
 152
 153
 154
 155
 156
 157
 158
 159
 160
 161
 162
 163
 164
 165
 166
 167
 168
 169
 170
 171
 172
 173
 174
 175
 176
 177
 178
 179
 180
 181
 182
 183
 184
 185
 186
 187
 188
 189
 190
 191
 192
 193
 194
 195
 196
 197
 198
 199
 200
 201
 202
 203
 204
 205


In [39]:
len(data['AcqDate'].values)

48211

In [488]:
# fico=[]
# for item in data['AcqDate'].values:
#     if type(item) == int:
#         if len(str(item))<=5:
#             fico.append(pd.to_datetime(DT.date(1900,1,1) + DT.timedelta(days=item-2)))
#     if type(item)==datetime:
#         fico.append(pd.to_datetime(item))
#     if type(item)==int:
#         if len(str(item))==8:
#             fico.append(pd.to_datetime(str(item)))
#     if type(item)==str:
#         if len(str(item))==10:
#             fico.append(pd.to_datetime(item))
#     if len(str(item))==3:
#         fico.append(item)
# #     if len(str(item))==1:
# #         fico.append(pd.to_datetime((item)))
                    
# len(fico)

2745

In [65]:
data['AcqDate'].iloc[2313]

'37275'

In [35]:
# data['AcqDate'][2313]=pd.to_datetime(DT.date(1900,1,1) + DT.timedelta(days=data['AcqDate'].iloc[2313]-2))
date=data['AcqDate']

In [117]:
strr='21-Feb-02; 25-Mar-02'
strr=' '.join(strr.split(';',1)[1].split())
strr

'25-Mar-02'

In [49]:

pd.DataFrame(data).to_excel('C:\\Users\\oluwaseun.johnson\\Desktop\\DATA VALIDATION\\CORRECTED\\DATE_.xlsx',header=True,index=False) #changes array to dataframe 1st then to excel
os.getcwd()

'C:\\Users\\oluwaseun.johnson\\Desktop\\DATA VALIDATION\\CORRECTED'

In [73]:
pd.to_datetime(37744)

Timestamp('1970-01-01 00:00:00.000037744')

In [195]:
pd.to_datetime('18 APR, 2014.')


Timestamp('2014-04-18 00:00:00')

In [8]:
# pd.to_datetime(38717)+pd.to_datetime('19700101')
import datetime as DT
DT.date(1900,1,1) + DT.timedelta(days=38728-2)
# len('20080926')

datetime.date(2006, 1, 11)

In [93]:
pd.to_datetime(DT.date(1900,1,1) + DT.timedelta(days=int('37744')-2))

Timestamp('2003-05-03 00:00:00')

In [165]:
# len(original_extract)
data_original.groupby('ClientCode')['S/N'].nunique()

ClientCode
           8127
AD01       2745
AE47        159
AM03        474
CV06      48211
EF14       7692
ER44        164
ES13      14810
FH15        217
FP43        128
FR17        396
FR45        338
LK18         18
MB19      12417
MN20         32
NC23        109
ND25        112
NR42         43
NS16        606
NW24        996
OP46        183
OW28         25
PN29       7267
SA32       2109
SE34      25688
SE42        103
SH31       2009
ST38        811
WAEP49       63
ZB41        143
Name: S/N, dtype: int64

In [ ]:
pd.read_csv('Z:\\Mr Biola\\CORRECTED DB DATA\\SW34 TO BE FIXED.csv')

In [ ]:
da['MediaNo']

In [5]:
da=data_original.loc[data_original['ClientCode']==filename]
len(da)
# da=da.str.strip()
da['TapeLocation']=da['TapeLocation'].str.strip()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [74]:
DA=pd.read_excel('Z:\\SJ\\CLIENT MERGING.XLSX')

In [44]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
# data_original.columns

# data_original.groupby('TapeAvailability')['S/N'].nunique()
# data_original[data_original.TapeAvailability==(like='NOT')]
# NOT AVAILABLE
# data_original[data_original.TapeAvailability.filter(like='NOT')]
# da.groupby('TapeAvailability')['S/N'].nunique()
da.groupby('TapeLocation')['S/N'].nunique()

NameError: name 'da' is not defined

In [7]:
# pd.options.display.max_rows = None
# pd.options.display.max_columns = None
# da[da.StorageMedium == 'N/A']
# da.groupby('TapeAvailability')['S/N'].nunique()
# da[da.TapeLocation.str.contains('-- -', regex=False)]
# type(da.TapeLocation)
# da[(' '.join(da['TapeLocation'].split())!='') & (da['TapeLocation']!='----')]
# arr=[103942,103945,103982,103979,104040,104045,104095,104098,104178,104179,104180,104187,104190,104226,104227,104274]
# da[(da['MediaNo'].isin(arr))][['TapeLocation','MediaNo']]
da[(da['TapeLocation']=='')]['MediaNo']

100267    J03149
100268    J03149
100269    J03149
100270    J03149
100271    J03149
100272    J03149
100273    J03149
100274    J03149
100275    J03149
100276    J03149
100277    J03149
100279    J03150
100280    J03150
100281    J03150
100282    J03150
100283    J03150
100284    J03150
100285    J03150
100286    J03150
100287    J03150
100288    J03150
100289    J03150
100291    J03151
100292    J03151
100293    J03151
100294    J03151
100295    J03151
100296    J03151
100297    J03151
100298    J03151
           ...  
101485    I04271
101486    I04271
101487    I04271
101488    I04271
101489    I04271
101490    I04271
101491    I04271
101492    I04271
101493    I04271
101494    I04271
101495    I04271
101496    I04271
101497    I04271
101498    I04271
101499    I04271
101500    I04271
101501    I04271
101502    I04271
101503    I04271
101504    I04271
101505    I04271
101506    I04271
101507    I04271
101508    I04271
101509    I04271
101510    I04271
101511    I04271
101512    I042

In [226]:

# df_obj.apply(lambda x: x.str.strip())

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [151]:
pd.options.display.max_rows = 50
pd.options.display.max_columns = 50

In [45]:
da['StorageMedium'].iloc[395]

'3590B'

In [24]:
for item in da['StorageMedium']:
    if item=='':
        print(item)

In [11]:
count=0
for item in da['TapeLocation']:
    if 'N/A' in item:
        da['TapeLocation'].iloc[count]='NaN'
        count=count+1
    else:
        count=count+1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
da.dropna(subset=['TapeLocation'],inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
final_data=da[da.TapeLocation != 'NaN']

In [135]:
# data_uploaded.loc[(data_uploaded['ClientCode']) & (data_uploaded['TapeAvailability']=='Not Available') ,'MediaNo']

Series([], Name: MediaNo, dtype: object)

In [131]:
# data_uploaded.TapeAvailability

0         AVAILABLE
1         AVAILABLE
2         AVAILABLE
3         AVAILABLE
4         AVAILABLE
5         AVAILABLE
6         AVAILABLE
7         AVAILABLE
8         AVAILABLE
9         AVAILABLE
10        AVAILABLE
11        AVAILABLE
12        AVAILABLE
13        AVAILABLE
14        AVAILABLE
15        AVAILABLE
16        AVAILABLE
17        AVAILABLE
18        AVAILABLE
19        AVAILABLE
20        AVAILABLE
21        AVAILABLE
22        AVAILABLE
23        AVAILABLE
24        AVAILABLE
25        AVAILABLE
26        AVAILABLE
27        AVAILABLE
28        AVAILABLE
29        AVAILABLE
            ...    
136318          N/A
136319          N/A
136320          N/A
136321          N/A
136322          N/A
136323          N/A
136324          N/A
136325          N/A
136326          N/A
136327          N/A
136328          N/A
136329          N/A
136330          N/A
136331          N/A
136332          N/A
136333          N/A
136334          N/A
136335          N/A
136336          N/A


In [136]:
# for unq in pd.unique(data_uploaded.TapeAvailability):
#     print(unq)

In [4]:
for unq in pd.unique(data_uploaded['ClientCode']):
    print(unq)

AD01
AE47
DB11
DP10
EF14
ER44
FP43
ES13
FR45
FZ45
LK18
MB19
MN20
NC23
NW24
NR42
ND25
OP46
SH31
SA32
ST38
CV06
AM03
NS16
PN29
SW34
FH15
FR17
WAEP49
ZB41
N/A


In [ ]:
for unq in pd.unique(data_original['ClientCode']):
    print(unq)

In [8]:
len(original_extract)

2109

In [7]:
len(upload_extract)

1868

In [11]:
len(extra_extract)

0

In [12]:
data_original.loc[data_original['ClientCode']==filename,'MediaNo'].size

48211

In [42]:
dddd=data_original.loc[data_original['ClientCode']==filename,'MediaNo']
len(dddd)
dddd.values

array(['OML 85_001', 'OML 85_002', 'OML 85_003', 'OML 85_004',
       'OML 85_005', 'OML 85_006', 'OML 85_007', 'OML 85_008',
       'OML 85_009', 'OML 85_010', 'OML 85_011', 'OML 85_012',
       'OML 85_013', 'OML 85_014', 'OML 85_015', 'OML 85_016',
       'OML 85_017', 'OML 85_018', 'OML 85_019', 'OML 85_020',
       'OML 85_021', 'OML 85_022', 'OML 85_023', 'OML 85_024',
       'OML 85_025', 'OML 85_026', 'OML 85_027', 'OML 85_028', '7764000-1',
       '7764000-2', 'HDD', 'FE160001', 'FE160002', 'FE160003', 1001, 1002,
       1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013,
       1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024,
       1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035,
       1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046,
       1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057,
       1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068,
       1069, 1070, 1071, 1072, 10

In [43]:
na=[]
for item in data_original[(data_original['ClientCode']==filename) & (data_original['MediaNo']=='N/A')]['MediaNo']:
    na.append(item)
na
# data_original[(data_original['ClientCode']==filename) & (data_original['MediaNo']=='N/A')]

[]

In [44]:
a=dddd[data_original.loc[data_original['ClientCode']==filename,'MediaNo'].duplicated(keep=False)]
# data_original[m.duplicated(keep=False)]
# a=pd.DataFrame(a)
len(a)



0

In [15]:
data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo'].size

34062

In [45]:
uuuu=data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo']

In [46]:
# b=data_uploaded[data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo'].duplicated(keep=False)]['MediaNo']
b=uuuu[data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo'].duplicated(keep=False)]
# b=pd.DataFrame(b)
# b.values #dataframe to array
# b=[]
# for item, count in collections.Counter(data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo'].values).items():
#     if count>1:
#         b.append(item)
len(b)


136

In [23]:
# b.subtract(a)
# data_uploaded.subtract(data_original)
# original=[]
# uploaded=[]
# original_dup=[]
# original_no_dup=[]
# uploaded_dup=[]
# uploaded_no_dup=[]
# for item in data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo']:
#     uploaded.append(item)
# for item in data_original.loc[data_original['ClientCode']==filename,'MediaNo']:
#     original.append(item)
for item in upload_extract:
    if item not in original_extract:
        print(item) #comparing data in uploaded and are not in array

84946
NULL
4058
4057
4043
4053
4042
4059
4056
4052


In [150]:
# data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo'].values.subtract(data_original.loc[data_original['ClientCode']==filename,'MediaNo'].values)
# data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo'].values

In [47]:
yo=[]
for item in b.values:
    if item in a.values:
        yo.append(item)
len(yo)

0

In [152]:
# for item in pd.unique(data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo']):
#     if item not in pd.unique(data_original.loc[data_original['ClientCode']==filename,'MediaNo']):
#         print(item) #comparing unique data in uploaded and are not in array

In [48]:
left=[]
for item, count in collections.Counter(data_original.loc[data_original['ClientCode']==filename,'MediaNo']).items():
    if count<=1:
        left.append(item)
len(left) #check for unique items


128

In [49]:
right=[]
for item, count in collections.Counter(data_uploaded.loc[data_uploaded['ClientCode']==filename,'MediaNo']).items():
    if count<=1:
        right.append(item)
len(right) #check for unique items


0

In [51]:
fi=[]

for item in data_original.loc[data_original['ClientCode']==filename,'MediaNo'].values:
    if item not in yo+right:
        fi.append(item)
len(fi)
for item in data_original.loc[data_original['ClientCode']==filename,'MediaNo'].values:
    if item not in yo+right+fi:
        print(item)
len(yo+fi+right+na)
final_data=data_original[(data_original['ClientCode']==filename) & (data_original['MediaNo'].isin(yo+fi+right+na))] #is in some iterable values
len(final_data['MediaNo'])

#change dataframe fields position: df=  pd.DataFrame(data,columns=["a","b","c"]); rearrange:df=df[["c","a","b"]]

128

In [27]:
len(final_data['MediaNo'])

48211

In [52]:
for item in final_data['MediaNo'].values:
    if item not in data_original.loc[data_original['ClientCode']==filename,'MediaNo'].values:
        print(item)

In [209]:
# #dictionary to csv
# with open ('d.csv') as csv_file:
#     writer=csv.writer(csv_file)
#     for key, value in extract.items():  #DICTIONAARY IS extract
#         writer.writerow([key,value])
# csv_file.close()

#array to excel
# pd.DataFrame(fraz).to_excel('C:\\Users\\oluwaseun.johnson\\Desktop\\DATA VALIDATION\\CORRECTED\\'+filename+'.xlsx',header=True,index=False) #changes array to dataframe 1st then to excel
# pd.DataFrame(final_data).to_excel('C:\\Users\\Leo-SJ\\Desktop\\DATA VALIDATION\\CORRECTED\\'+filename+'.xlsx',header=True,index=False) #changes array to dataframe 1st then to excel

pd.DataFrame(ext).to_excel('C:\\Users\\oluwaseun.johnson\\Desktop\\DATA VALIDATION\\CORRECTED\\'+'EXTRACT'+'.xlsx',header=True,index=False) #changes array to dataframe 1st then to excel

# ORIGINAL DATA

In [58]:
data_original_no_dups=[]
for item, count in collections.Counter(data_original['MediaNo']).items():
    if count==1:
        data_original_no_dups.append(item)
len(data_original_no_dups)
data_original_no_dups 

['A019734',
 'A019735',
 '8403-999',
 '8403-997',
 '8403-998',
 '8403-996',
 '8403-994',
 '8403-995',
 '8403-993',
 '8403-992',
 '8403-990',
 '8403-991',
 '8403-99',
 '8403-988',
 '8403-989',
 '8403-987',
 '8403-986',
 '8403-984',
 '8403-985',
 '8403-983',
 '8403-981',
 '8403-982',
 '8403-980',
 '8403-98',
 '8403-978',
 '8403-979',
 '8403-977',
 '8403-975',
 '8403-976',
 '8403-974',
 '8403-973',
 '8403-971',
 '8403-972',
 '8403-970',
 '8403-969',
 '8403-97',
 '8403-968',
 '8403-967',
 '8403-965',
 '8403-966',
 '8403-964',
 '8403-962',
 '8403-963',
 '8403-961',
 '8403-960',
 '8403-959',
 '8403-96',
 '8403-958',
 '8403-956',
 '8403-957',
 '8403-955',
 '8403-954',
 '8403-952',
 '8403-953',
 '8403-951',
 '8403-95',
 '8403-950',
 '8403-949',
 '8403-948',
 '8403-946',
 '8403-947',
 '8403-945',
 '8403-943',
 '8403-944',
 '8403-942',
 '8403-941',
 '8403-94',
 '8403-940',
 '8403-939',
 '8403-937',
 '8403-938',
 '8403-936',
 '8403-935',
 '8403-933',
 '8403-934',
 '8403-932',
 '8403-930',
 '8403-

In [40]:
data_original_dups=[]
for item in data_original['MediaNo']:
    if item not in data_original_no_dups:
        data_original_dups.append(item)
len(data_original_dups)

177

In [41]:
len(data_original_no_dups)+len(data_original_dups)

14810

# UPLOADED DATA

In [42]:
data_uploaded_no_dups=[]
for item, count in collections.Counter(data_uploaded['MediaNo']).items():
    if count==1:
        data_uploaded_no_dups.append(item)
len(data_uploaded_no_dups)

14352

In [43]:
data_uploaded_dups=[]
for item in data_uploaded['MediaNo']:
    if item not in data_uploaded_no_dups:
        data_uploaded_dups.append(item)
len(data_uploaded_dups)
# data_uploaded_dups=list(set(data_uploaded_no_dups).difference(data_uploaded['MediaNo']))
# len(data_uploaded_dups)

645

In [44]:
len(data_uploaded_no_dups)+len(data_uploaded_dups)

14997

# COMPARISM

In [61]:
d_final=[]
for item in data_uploaded_no_dups + data_uploaded_dups:
    if item not in data_original_no_dups + data_original_dups:
        d_final.append(item)
len(d_final)
d_final

[nan,
 'VEH0341',
 'VEH0342',
 'VEH0343',
 'VEH0344',
 'VEH0345',
 'VEH0346',
 'VEH0347',
 'VEH0348',
 'VEH0349',
 'VEH0350',
 'VEH0351',
 'VEH0352',
 'VEH0353',
 'VEH0354',
 'VEH0355',
 'VEH0356',
 'VEH0357',
 'VEH0358',
 'VEH0359',
 'VEH0360',
 'VEH0361',
 'VEH0362',
 'VEH0363',
 'VEH0364',
 'VEH0365',
 'VEH0366',
 'VEH0367',
 'VEH0368',
 'VEH0369',
 'VEH0370',
 'VEH0371',
 'VEH0372',
 'VEH0373',
 'VEH0374',
 'VEH0375',
 'VEH0376',
 'VEH0377',
 'VEH0378',
 'VEH0379',
 'VEH0380',
 'VEH0381',
 'VEH0382',
 'VEH0383',
 'VEH0384',
 'VEH0385',
 'VEH0386',
 'VEH0387',
 'VEH0388',
 'VEH0389',
 'VEH0390',
 'VEH0391',
 'VEH0392',
 'VEH0393',
 'VEH0394',
 'VEH0395',
 'VEH0396',
 'VEH0397',
 'VEH0398',
 'VEH0399',
 'VEH0400',
 'VEH0401',
 'VEH0402',
 'VEH0403',
 'VEH0404',
 'VEH0405',
 'VEH0406',
 'VEH0407',
 'VEH0408',
 'VEH0409',
 'VEH0410',
 'VEH0411',
 'VEH0412',
 'VEH0413',
 'VEH0414',
 'VEH0415',
 'VEH0416',
 'VEH0417',
 'VEH0418',
 'VEH0419',
 'VEH0420',
 'VEH0421',
 'VEH0422',
 'VEH0423'

In [46]:
d_extra=[]
for item in d_final:
    if item in data_extra['MediaNo']:
        d_extra.append(item)
len(d_extra)

0

In [47]:
data_extra=[]
data_needed=[]
for item in data_uploaded_dups:
    if item not in data_original_dups:
        data_extra.append(item)
    else:
        data_needed.append(item)
len(data_extra)
# data_extra=list(set(data_uploaded_dups).difference(data_original_dups))
# len(data_extra)

480

In [48]:
len(data_needed)

165

In [49]:
data_uploaded_dups_corrected=[]
for item in data_original_no_dups:
    if item in data_uploaded_dups:
        data_uploaded_dups_corrected.append(item)
len(data_uploaded_dups_corrected)


240

In [50]:
len(data_uploaded_no_dups)+len(data_needed)+len(data_uploaded_dups_corrected)

14757

In [51]:
len(data_uploaded_no_dups)+len(data_uploaded_dups_corrected)

14592

In [52]:
for item, count in collections.Counter(data_original_no_dups + data_uploaded_dups_corrected).items():
    if count>1:
        print(item)
# for item in data_original_no_dups + data_uploaded_dups_corrected:
#     if item not in data_original_no_dups:
#         print(item)

CHB000559
CHB000560
CHB000561
CHB000562
CHB000563
CHB000564
CHB000565
CHB000566
CHB000567
CHB000568
CHB000569
CHB000570
CHB000571
CHB000572
CHB000573
CHB000574
CHB000575
CHB000576
CHB000577
CHB000578
CHB000579
CHB000580
CHB000581
CHB000582
CHB000583
CHB000584
CHB000585
CHB000586
CHB000587
CHB000588
CHB000589
CHB000590
CHB000591
CHB000592
CHB000593
CHB000594
CHB000595
CHB000596
CHB000597
CHB000598
CHB000599
CHB000600
CHB000601
CHB000602
CHB000603
CHB000604
CHB000605
CH000A677
CH000A678
26340
BSLT0001 (1 OF 106)
BSLT0002 (2 OF 106)
BSLT0003 (3 OF 106)
BSLT0004 (4 OF 106)
BSLT0005 (5 OF 106)
BSLT0006 (6 OF 106)
BSLT0007 (7 OF 106)
BSLT0008 (8 OF 106)
BSLT0009 (9 OF 106)
BSLT0010 (10 OF 106)
BSLT0011 (11 OF 106)
BSLT0012 (12 OF 106)
BSLT0013 (13 OF 106)
BSLT0014 (14 OF 106)
BSLT0015 (15 OF 106)
BSLT0016 (16 OF 106)
BSLT0017 (17 OF 106)
BSLT0018 (18 OF 106)
BSLT0019 (19 OF 106)
BSLT0020 (20 OF 106)
BSLT0021 (21 OF 106)
BSLT0022 (22 OF 106)
BSLT0023 (23 OF 106)
BSLT0024 (24 OF 106)
BSLT0025 

In [4]:
import os
os.chdir('C:\\Users\\oluwaseun.johnson\\Desktop\\DATA VALIDATION')
os.getcwd()

'C:\\Users\\oluwaseun.johnson\\Desktop\\DATA VALIDATION'

In [63]:
data_source_original.loc[data_source_original['MediaNo']=='A019734','ClientCode']


3572    ES13
Name: ClientCode, dtype: object

In [62]:
data_source_uploaded.loc[data_source_original['MediaNo']=='VEH0472','ClientCode']

Series([], Name: ClientCode, dtype: object)

In [67]:
len(data_source_original.loc[data_source_original['ClientCode']=='AD01','MediaNo'])

2745

In [68]:
len(data_source_uploaded.loc[data_source_uploaded['ClientCode']=='AD01','MediaNo'])

3040

In [72]:
d=[]
d1=[]
for item in data_source_uploaded.loc[data_source_uploaded['ClientCode']=='AD01','MediaNo']:
    d.append(item)
for item in data_source_original.loc[data_source_original['ClientCode']=='AD01','MediaNo']:
    d1.append(item)
#     if item in data_source_original.loc[data_source_original['ClientCode']=='AD01','MediaNo']:
#         print(item)

In [75]:
for item in d:
    if item not in d1:
        print(item)

# DATE VALIDATION

In [4]:
import os
import pandas as pd
import pprint
import csv
import collections
import numpy as np

# DATA MINING FROM WEB PAGES USING BEAUTIFULSOUP (VOLUNTEER PROJECT)

In [47]:
import urllib.request
from bs4 import BeautifulSoup
# url='https://doublethedonation.com/tips/companies-that-donate-to-nonprofits/'
url='http://fortune.com/2016/06/22/fortune-500-most-charitable-companies/'

In [56]:
page=urllib.request.urlopen(url)
soup=BeautifulSoup(page,'html.parser')
title=soup.find_all('strong')
p=soup.find_all('p')

In [57]:
for i in p:
    print(i.get_text())

By surveying the top 150 companies in the Fortune 500 and collecting data from public documents filed with the IRS and SEC, The Chronicle of Philanthropy has published a list of the most charitable public companies, which can be viewed here. With help from the Chronicle, Fortune has identified the top 20 most generous companies, as measured by cash contributions. The dollar amounts cited below were independently verified by Fortune. In total, the 20 most generous companies donated $3.5 billion in cash in 2015. 
1. Gilead Sciences
 
Gilead’s 2015 cash contributions: $446.7 million
Gilead’s (gild) giving jumped by nearly two-thirds in 2015, due to rising revenues as the biotech firm expanded into the Hepatitis C market. Most of the company’s grants are reactive, based on specific funding requests from nonprofit groups. The company declined to provide an accounting of its biggest cash grants in 2015, but said that most giving falls within its work on HIV/AIDS and liver disease. Last year,

In [39]:
page=urllib.request.urlopen(url)
soup=BeautifulSoup(page,'html.parser')
title=soup.find_all('h3', attrs={'class':'av-special-heading-tag'})
voluntary_title=soup.find_all('h3', attrs={'class':'iconbox_content_title'})
paragraph=soup.find_all('div', attrs={'class':'iconbox_content'})

In [42]:
para=[]
for i in paragraph:
    para.append(i.get_text())
len(para)

112

In [6]:
tit=[]
tit_voluntary=[]
for i in title:
    tit.append(i.get_text())
for j in voluntary_title:
    tit_voluntary.append(j.get_text())

In [7]:
tit

['Matching Gifts',
 'Volunteer Grants',
 'Crowdfunding Campaign',
 'Bank of America',
 'General Electric',
 'Google',
 'Darden Restaurant Group',
 'General Mills',
 'BP',
 'State Street',
 'Intuit',
 'McAfee',
 'Chevron',
 'Disney',
 'Starbucks',
 'Outerwall',
 'CVS',
 'Best Buy',
 'Kohl’s',
 'Verizon',
 'CarMax',
 'IBM',
 'Liberty Global',
 'Microsoft',
 'Teradata',
 'Kaiser Permanente',
 'IMF (International Monetary Fund)',
 'American Airlines',
 'Delta Airlines',
 'Southwest Airlines',
 'United Airlines',
 'JetBlue',
 'Alaska Airlines',
 'Hilton Worldwide',
 'InterContinental Hotels Group',
 'SeaWorld',
 'Universal',
 'Host Hotels',
 'Choice Hotels',
 'Amazon',
 'Costco',
 'The Home Depot',
 'State Farm Insurance',
 'Target',
 'Walmart',
 'JC Penney',
 'Office Depot',
 'Dick’s Sporting Goods',
 'Gap',
 'Macy’s',
 'Reebok',
 'Nordstrom',
 'Albertson’s',
 'Coca-Cola',
 'Kroger',
 'Supervalu',
 'Whole Foods',
 'Publix',
 'The Fresh Market',
 'Guide to Matching Gifts',
 'Guide to Volunt

In [8]:
len(tit)

62

In [12]:
tit_voluntary

['Bank of America’s Matching Gift Program',
 'Bank of America Charitable Foundation',
 'General Electric’s Matching Gift Program',
 'The GE Foundation',
 'Google’s Matching Gift Program',
 'Google Dot Org',
 'Darden’s Matching Gift Program',
 'Darden Harvest Program',
 'General Mills’ Matching Gift Program',
 'General Mills’ Global Responsibility Initiatives',
 'BP’s Fundraising Match Program',
 'BP Foundation',
 'State Street’s Fundraising Match Program',
 'State Street’s Corporate Citizenship Program',
 'Intuit’s Fundraising Match Program',
 'Intuit’s Matching Gift Program',
 'McAfee’s Fundraising Match Program',
 'McAfee’s Matching Gift Program',
 'Chevron’s Volunteer Grant Program',
 'Chevron’s Corporate Social Responsibility Program',
 'Disney’s Volunteer Grant Program',
 'Disney’s Corporate Philanthropy Program',
 'Starbucks’ Volunteer Grant Program',
 'Starbucks’ Corporate Philanthropy Program',
 'Outerwall’s Volunteer Grant Program',
 'Outerwall’s Matching Gift Program',
 'CVS’

In [45]:
dic={'company_name': [], 'foundation_name': [],'foundation_program': [],'notes': []}
j=0
for i in range(3,59):
    dic['company_name'].append(tit[i])
    dic['foundation_name'].append(tit_voluntary[j])
    dic['foundation_program'].append(tit_voluntary[j+1])
    dic['notes'].append(para[j+1])
    j=j+2

In [29]:
dic

{'company_name': ['Bank of America',
  'General Electric',
  'Google',
  'Darden Restaurant Group',
  'General Mills',
  'BP',
  'State Street',
  'Intuit',
  'McAfee',
  'Chevron',
  'Disney',
  'Starbucks',
  'Outerwall',
  'CVS',
  'Best Buy',
  'Kohl’s',
  'Verizon',
  'CarMax',
  'IBM',
  'Liberty Global',
  'Microsoft',
  'Teradata',
  'Kaiser Permanente',
  'IMF (International Monetary Fund)',
  'American Airlines',
  'Delta Airlines',
  'Southwest Airlines',
  'United Airlines',
  'JetBlue',
  'Alaska Airlines',
  'Hilton Worldwide',
  'InterContinental Hotels Group',
  'SeaWorld',
  'Universal',
  'Host Hotels',
  'Choice Hotels',
  'Amazon',
  'Costco',
  'The Home Depot',
  'State Farm Insurance',
  'Target',
  'Walmart',
  'JC Penney',
  'Office Depot',
  'Dick’s Sporting Goods',
  'Gap',
  'Macy’s',
  'Reebok',
  'Nordstrom',
  'Albertson’s',
  'Coca-Cola',
  'Kroger',
  'Supervalu',
  'Whole Foods',
  'Publix',
  'The Fresh Market'],
 'foundation_name': ['Bank of America’

In [46]:
import xlsxwriter

workbook = xlsxwriter.Workbook('d_charity.xlsx')
worksheet = workbook.add_worksheet()


row = 0
col = 0

for key in dic.keys():
    row =0
    worksheet.write(row, col, key)
    row+=1
    for item in dic[key]:
        worksheet.write(row, col + 1, item)
        row += 1
    col+=1

workbook.close()

In [ ]:
col = 0

for key in d.keys():
    row = 0
    worksheet.write(row, col, key)
    row += 1
    for item in d[key]:
        worksheet.write(row, col, item)
        row += 1
    col += 1

workbook.close()

In [59]:
import os
os.startfile('C:\\Users\\Leo-SJ\\Documents\\MASTERS\\NIR_admin.kmz')


# EXTRA PROJECTS USING BEAUTIFULSOUP

In [1]:
import urllib.request
from bs4 import BeautifulSoup
# url='https://doublethedonation.com/tips/companies-that-donate-to-nonprofits/'
url='https://printivo.com/'

In [ ]:
page=urllib.request.urlopen(url)
soup=BeautifulSoup(page,'html.parser')
title=soup.find_all('strong')
p=soup.find_all('p')

In [ ]:
for i in p:
    print(i.get_text())

In [37]:
page=urllib.request.urlopen(url)
soup=BeautifulSoup(page,'html.parser')
title=soup.find_all('a', attrs={'class':'text-sm block py-1 transition-2'})

tit2=[]
for i in title:
    tit2.append(i.get_text())
tit
title
t=soup.find_all('a', attrs={'class':'text-sm block py-1 transition-2'})[1]['data-slug']
obj2=soup.find("div", attrs={'class':'w-full main-nav-parent-items'})
children2 = obj2.findChildren("a" , attrs={'class':'text-sm block py-1 transition-2'},recursive=False)

tit=[]
for i in range(len(tit2)):
    tit.append(soup.find_all('a', attrs={'class':'text-sm block py-1 transition-2'})[i]['data-slug'])
tit

['banners-large-format',
 'branded-envelopes',
 'branded-notepads-and-jotters',
 'business-cards',
 'calendars',
 'caps-hats',
 'corporate-gifts',
 'custom-mugs',
 'custom-t-shirts',
 'flyers-handbills',
 'frames-wall-arts',
 'greeting-cards',
 'letterhead',
 'marketing-brochures',
 'paper-bags',
 'phone-cases',
 'plastic-identity-cards',
 'posters',
 'presentation-folders',
 'stickers',
 'tote-bags',
 'wedding-stationery']

In [40]:
data=[]
for name in tit:
    obj=soup.find("div", {"id": name+"-sub-menu"})
    children = obj.findChildren("a" , attrs={'class':'text-sm mb-2 block'})
obj

<div class="hidden invisible transition-5" id="wedding-stationery-sub-menu"><a class="text-sm mb-2 block" data-price="35000" data-quantity="100" data-thumb="/files/category_thumbnails/1446914098_Wedding-Invitation.png" data-title="Wedding Invitations" href="/category/wedding-invitations">Wedding Invitations</a><a class="text-sm mb-2 block" data-price="17500" data-quantity="100" data-thumb="/files/category_thumbnails/1446911491_safethedate.png" data-title="Save The Date (A6 sizes)" href="/category/save-the-date-a6-sizes">Save The Date (A6 sizes)</a><a class="text-sm mb-2 block" data-price="9499" data-quantity="100" data-thumb="/files/category_thumbnails/1446704368_Food Menu2.png" data-title="Food &amp; Drink Menus" href="/category/food-drink-menus">Food &amp; Drink Menus</a></div>

In [15]:
for i in children:
    print(i.get_text())

Roll Up Banners (Big Base)
X-Banner
Rollup Banner (Small Base)
Flex Banners Medium (5ft x 3ft)
Flex Banners Large (7ft x 3ft)
Teardrop Banners (3ft by 8ft)
A-Frames Signs
Backdrop Banners
Instagram Frame Boards
Dummy Cheque
Snapper Frame
Waterbase Poster Signs


In [ ]:
obj=soup.find("div", {"id": "Banners & Large Format-sub-menu"})


# JSON WRANGLING

In [48]:
import json
import os
import pprint
# import objectpath
import pandas as pd
os.getcwd()

'C:\\Users\\Leo-SJ\\Documents\\MASTERS'

In [52]:
path='C:\\Users\\Leo-SJ\\Desktop\\DATA VALIDATION\\CODE\\CODE\\JSON\\'
READ=pd.read_excel("C:\\Users\\Leo-SJ\\Desktop\\DATA VALIDATION\\CODE\\CODE\\JSON\\ObjectRenamed.xlsx",sheet_name="Sheet2")
# READ['Name']
READ[READ['Name']=='Object ID 19'].index.values.astype(int)[0]   #get index based on value in array
# READ.iloc[2]['Replace']  #get location of column replace based on index
# READ['Name']=='Object ID 19'
# READ['Name'].values
# READ.columns

2

In [53]:
json_value=json.load(open(path+'KML.json'))
# print(json.dumps(json_value['features'],indent=1,separators=(', ', ': ')))
# json_value['features'][1]['properties']['Name']
# print(json_value['features'])
json_value['features'][0]

{'type': 'Feature',
 'properties': {'Name': 'OPEN', 'Description': '', 'Prospect': 'OPL301'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[3.139307883338614, 6.89705072588582, 0.0],
    [3.138878182211314, 6.808961994789312, 0.0],
    [3.138018779956714, 6.79521155871571, 0.0],
    [3.136729676574814, 6.78403932940591, 0.0],
    [3.135440573192914, 6.775875007987209, 0.0],
    [2.792539073607482, 6.778453214751009, 0.0],
    [2.791679671352882, 6.809391695916612, 0.0],
    [2.786952958952582, 6.811540201553113, 0.0],
    [2.782655947679581, 6.813259006062312, 0.0],
    [2.776640131897381, 6.812399603807712, 0.0],
    [2.76503820146028, 6.812399603807712, 0.0],
    [2.755155075532379, 6.812829304935012, 0.0],
    [2.743553145095277, 6.820563925226413, 0.0],
    [2.736677927058476, 6.833454959045414, 0.0],
    [2.734959122549277, 6.851072705264716, 0.0],
    [2.734959122549277, 6.862674635701817, 0.0],
    [2.739685834949577, 6.871698359375118, 0.0],
    [2.746561052986378, 6.879

In [76]:
with open('KML.json', 'rb') as f:
    data = json.load(f)
    count=0
    for i in range(len(data['features'])):
        if data['features'][i]['properties']['Name'] in READ['Name'].values:
            location=READ[READ['Name']==data['features'][i]['properties']['Name']].index.values.astype(int)[0]
            data['features'][i]['properties']['Name']=READ.iloc[location]['Company Name']
            data['features'][i]['properties']['Prospect']=READ.iloc[location]['Replace']
#             print(READ['Name'])
f.close()
jsonFile = open("SJ_SPECIAL.json", "w+")
jsonFile.write(json.dumps(data))
jsonFile.close()
#     json_value['features'][1]['properties']['Name']='SEUN'
#     data['id'] = 134 # <--- add `id` value.
#     f.seek(0)        # <--- should reset file position to the beginning.
#     json.dump(data, f, indent=4)
#     f.truncate()     # remove remaining part
data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'Name': 'OPEN', 'Description': '', 'Prospect': 'OPL301'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[3.139307883338614, 6.89705072588582, 0.0],
      [3.138878182211314, 6.808961994789312, 0.0],
      [3.138018779956714, 6.79521155871571, 0.0],
      [3.136729676574814, 6.78403932940591, 0.0],
      [3.135440573192914, 6.775875007987209, 0.0],
      [2.792539073607482, 6.778453214751009, 0.0],
      [2.791679671352882, 6.809391695916612, 0.0],
      [2.786952958952582, 6.811540201553113, 0.0],
      [2.782655947679581, 6.813259006062312, 0.0],
      [2.776640131897381, 6.812399603807712, 0.0],
      [2.76503820146028, 6.812399603807712, 0.0],
      [2.755155075532379, 6.812829304935012, 0.0],
      [2.743553145095277, 6.820563925226413, 0.0],
      [2.736677927058476, 6.833454959045414, 0.0],
      [2.734959122549277, 6.851072705264716, 0.0],
      [2.734959122549277, 6.862674635701817, 0.0]

In [28]:
json_value['features'][1]['properties']['Name']

'ALFRED JAMES'

In [59]:
print ("json file content-",json_value)

json file content- {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'Name': 'OPEN', 'Description': '', 'Prospect': 'OPL301'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[3.139307883338614, 6.89705072588582, 0.0], [3.138878182211314, 6.808961994789312, 0.0], [3.138018779956714, 6.79521155871571, 0.0], [3.136729676574814, 6.78403932940591, 0.0], [3.135440573192914, 6.775875007987209, 0.0], [2.792539073607482, 6.778453214751009, 0.0], [2.791679671352882, 6.809391695916612, 0.0], [2.786952958952582, 6.811540201553113, 0.0], [2.782655947679581, 6.813259006062312, 0.0], [2.776640131897381, 6.812399603807712, 0.0], [2.76503820146028, 6.812399603807712, 0.0], [2.755155075532379, 6.812829304935012, 0.0], [2.743553145095277, 6.820563925226413, 0.0], [2.736677927058476, 6.833454959045414, 0.0], [2.734959122549277, 6.851072705264716, 0.0], [2.734959122549277, 6.862674635701817, 0.0], [2.739685834949577, 6.871698359375118, 0.0], [2.746561052986378, 6.8798626807938

In [18]:
with open("KML.json") as dfile:
    data=json.load(dfile)

In [60]:
json_tree=objectpath.Tree(data['features'])

type(json_tree)

objectpath.core.interpreter.Tree

In [55]:

result=tuple(json_tree.execute('$..ProspectName'))
result

('OPL303',
 'OPL302',
 'OPL304',
 'OPL305',
 'OPL306',
 'OPL307',
 'OPL474',
 'OPL308',
 'OPL202',
 '203',
 'OPL201',
 'OPL915',
 'OPL901')

In [54]:
for i in tuple(json_tree.execute('$..Name')):
    print(tuple(json_tree.execute('$..ProspectName')))

('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL20

('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL20

('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL20

('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL201', 'OPL915', 'OPL901')
('OPL303', 'OPL302', 'OPL304', 'OPL305', 'OPL306', 'OPL307', 'OPL474', 'OPL308', 'OPL202', '203', 'OPL20

In [26]:
result

('OPEN',
 'ALFRED JAMES',
 'OPEN',
 'OPEN',
 'CROWN WELL PETROLEUM',
 'CROWN WELL PETROLEUM',
 'OPEN',
 'OPEN',
 'OPEN',
 'OPEN',
 'OPEN',
 'OPEN',
 'ORIENTAL ENERGY',
 'OPEN',
 'Object ID 55',
 'Object ID 58',
 'Object ID 61',
 'Object ID 64',
 'Object ID 67',
 'Object ID 70',
 'Object ID 73',
 'Object ID 76',
 'Object ID 79',
 'Object ID 82',
 'Object ID 85',
 'Object ID 88',
 'Object ID 91',
 'Object ID 94',
 'Object ID 97',
 'Object ID 100',
 'Object ID 103',
 'Object ID 106',
 'Object ID 109',
 'Object ID 112',
 'Object ID 115',
 'Object ID 118',
 'Object ID 121',
 'Object ID 124',
 'Object ID 127',
 'Object ID 130',
 'Object ID 133',
 'Object ID 136',
 'Object ID 139',
 'Object ID 142',
 'Object ID 145',
 'Object ID 148',
 'Object ID 151',
 'Object ID 154',
 'Object ID 157',
 'Object ID 160',
 'Object ID 163',
 'Object ID 166',
 'Object ID 169',
 'Object ID 172',
 'Object ID 175',
 'Object ID 178',
 'Object ID 181',
 'Object ID 184',
 'Object ID 187',
 'Object ID 190',
 'Object I

In [101]:
with open('KML.json','r+') as f:
    data=json.load(f)
    for k, v in data.items():
        print()

# DATA MINING FROM TWITTER USING TWEEPY (VOLUNTEER PROJECT)